Download Devfile json schema

In [8]:
import json
import requests

schemaUrl = "https://devfile.io/devfile-schemas/2.2.2.json"
response = requests.get(schemaUrl, allow_redirects=True)
content = response.content.decode("utf-8")
schema = json.loads(content)

Download Devfile validator from GitHub

In [9]:
import urllib.request as request
url = "https://raw.githubusercontent.com/yangcao77/library/validate-binary/main"
request.urlretrieve(url, "./validator")

('./validator', <http.client.HTTPMessage at 0x7fa8f523e370>)

Grant execute permissions to the validator binary

In [10]:
import subprocess
subprocess.run(["chmod", "+x", "./validator"]) 

CompletedProcess(args=['chmod', '+x', './validator'], returncode=0)

Define function to parse and validate a given devfile

In [11]:
from jsonschema import validate
import yaml

def parse_and_validate_devfile_url(url, schema):
    response = requests.get(url, allow_redirects=True)
    content = response.content.decode("utf-8")
    
    try:
        data = yaml.safe_load(content)
    except:
        return None
    
    if "schemaVersion" in data:
        schemaVersion = data["schemaVersion"]
        if schemaVersion == "2.0.0" or schemaVersion == "2.1.0" or schemaVersion == "2.2.0":
            try:
                validate(data, schema)
            except:
                return None
            return data
    return None

Define function to search all devfiles on GitHub

In [12]:
from github import Github
from github import Auth
import os

def search_github():

    access_token = os.environ["ACCESS_TOKEN"]

    # using an access token
    auth = Auth.Token(access_token)

    # pygithub object
    g = Github(auth=auth)

    codes = g.search_code(query="filename:devfile.yaml")
   
    print("Total number of files titled devfile.yaml after search: ", codes.totalCount, end="\n")
    print("-----")
   
    x = 0
    devfileDataList = []
    for code in codes:
        x += 1
        parsedDevfileData = parse_and_validate_devfile_url(code.download_url, schema)
        if parsedDevfileData is not None:
            devfileDataList.append(parsedDevfileData)
    return devfileDataList

In [13]:
devfileDataList = search_github()
len(devfileDataList)

Total number of files titled devfile.yaml after search:  1000
-----


629